In [ ]:
from matplotlib.pyplot import axis
import numpy as np
import pandas as pd
import mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from pyspark.sql import SparkSession

In [ ]:
# Construct Input sql query and save it as a dataframe

sql_query = '''
SELECT *
FROM (
  select 
case when a.CompanyID like 'CUST%' then replace(a.CompanyID,'CUST','' ) else a.CompanyID end as ComanyId,
a.OrderNumTwo,
a.ItemID as ItemId,
a.PlacedUnits
from cdb.orderfct as a
where 1=1
and a.OwnCompanyID=20
and a.TotPlacedRev>0
and a.OrderLineStatus!=90
and a.ItemId not like '%[^0-9]%'
and a.PlacedDttm<'2023-04-01'
and a.PlacedDttm>='2022-04-01'
) tmp
'''
result_df = spark.sql(sql_query)
result_df = result_df.repartition(1)


In [ ]:
import datetime
import os

# Get the current date
current_date = datetime.date.today().strftime("%Y-%m-%d")

# Define the output file name
output_filename = f"MBA_input_{current_date}.csv"

# Define the output path
output_path = f"dbfs:/mnt/input data for ml projects/Constructed Input/MBA/{output_filename}"

# Save the DataFrame as a CSV file
result_df.write.csv(output_path, header=True, mode="overwrite")

# Get a list of files in the "Constructed Input" folder
folder_path = "dbfs:/mnt/input data for ml projects/Constructed Input/MBA/"
file_list = dbutils.fs.ls(folder_path)

# Sort the files by modification time (most recent first)
sorted_files = sorted(file_list, key=lambda x: x.modificationTime, reverse=True)

# Keep only the latest five files
latest_files = sorted_files[:5]

# Delete files that are not in the latest five
for file in file_list:
    if file not in latest_files:
        dbutils.fs.rm(file.path)

In [ ]:
df=result_df.toPandas()
df.head(5)

/databricks/spark/python/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/databricks/spark/python/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/databricks/spark/python/pyspark/sql/pandas/utils.py:64: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pyarrow.__version__) < LooseVersion(minimum_pyarrow_version):
/databricks/spark/python/pyspark/sql/pandas/utils.py:64: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pyarrow.__version__) < LooseVersion(minimum_pyarrow_version):


,ComanyId,OrderNumTwo,ItemId,PlacedUnits
0,5017245,309994839,2010,1.0
1,1434713,309971148,16100,1.0
2,1236895,309977685,7980,1.0
3,5015805,309978875,2220,1.0
4,1177714,309883310,2900,2.0


In [ ]:
df.reset_index
df.columns=['CustomerId','Order#','ItemId','Quantity_Purchased']
df.dropna(axis=0,subset=['Order#'],inplace=True)

basket = (df.groupby(['Order#','ItemId'])['Quantity_Purchased']
.sum().unstack().reset_index().fillna(0).set_index('Order#'))

basket.head(5)


ItemId,1010,1017,1050,1051,11460,1150,1151,11560,11720,1210,12230,12237,12320,1250,1350,16002,16006,16100,16110,16130,16200,16206,16210,16216,1710,17760,17767,17800,17970,17977,1860,1900,1907,19130,191580,19170,2010,2016,2040,2051,...,7870,7900,7907,7980,8010,8015,8016,8017,8030,8500,8510,8520,8530,8700,8710,8720,8740,9100,9107,9110,9120,9140,9150,9160,9180,9190,9197,92076,9210,9217,9220,9227,9230,9237,9250,9257,9550,9557,9560,9580
Order#,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
309799563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
309835371,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
309835483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
309835512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
309835527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 1-HOT-ENCODED 

def encode_units(x):
    try:
        x = float(x)
        if x <= 0:
            return 0
        if x >= 1:
            return 1
    except (TypeError, ValueError):
        # Handle non-numeric values
        return 0
    
bk2=basket.applymap(encode_units)
bk2.describe()

ItemId,1010,1017,1050,1051,11460,1150,1151,11560,11720,1210,12230,12237,12320,1250,1350,16002,16006,16100,16110,16130,16200,16206,16210,16216,1710,17760,17767,17800,17970,17977,1860,1900,1907,19130,191580,19170,2010,2016,2040,2051,...,7870,7900,7907,7980,8010,8015,8016,8017,8030,8500,8510,8520,8530,8700,8710,8720,8740,9100,9107,9110,9120,9140,9150,9160,9180,9190,9197,92076,9210,9217,9220,9227,9230,9237,9250,9257,9550,9557,9560,9580
count,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.00000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,...,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.00000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000,24243.000000
mean,0.009817,0.006559,0.006311,0.005115,0.007507,0.013777,0.010436,0.009859,0.002351,0.002681,0.009240,0.006930,0.001980,0.008827,0.052964,0.095533,0.014767,0.029782,0.029576,0.004372,0.031019,0.005857,0.022769,0.004909,0.001774,0.046405,0.01617,0.006971,0.011055,0.001402,0.017118,0.013117,0.002475,0.005156,0.002887,0.002805,0.362703,0.044178,0.011343,0.009817,...,0.005074,0.007672,0.002062,0.031885,0.098668,0.039187,0.022934,0.022357,0.064472,0.003424,0.002351,0.001609,0.000907,0.004084,0.002269,0.002970,0.004125,0.003630,0.000990,0.005734,0.002599,0.005197,0.002970,0.000907,0.001897,0.010477,0.002722,0.003176,0.00561,0.002599,0.007672,0.002021,0.005321,0.002310,0.013695,0.003011,0.000330,0.000041,0.023388,0.009529
std,0.098597,0.080721,0.079193,0.071337,0.086321,0.116567,0.101624,0.098801,0.048433,0.051712,0.095681,0.082958,0.044454,0.093540,0.223966,0.293955,0.120622,0.169988,0.169417,0.065981,0.173373,0.076311,0.149171,0.069891,0.042079,0.210365,0.12613,0.083203,0.104561,0.037424,0.129715,0.113779,0.049688,0.071622,0.053658,0.052888,0.480790,0.205494,0.105902,0.098597,...,0.071050,0.087257,0.045368,0.175699,0.298221,0.194043,0.149698,0.147845,0.245597,0.058413,0.048433,0.040077,0.030111,0.063774,0.047578,0.054417,0.064094,0.060141,0.031449,0.075505,0.050912,0.071907,0.054417,0.030111,0.043519,0.101823,0.052107,0.056269,0.07469,0.050912,0.087257,0.044913,0.072753,0.048007,0.116223,0.054793,0.018163,0.006423,0.151136,0.097150
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [ ]:
bk3 = apriori(bk2, min_support=0.00001, use_colnames=True,max_len=2,low_memory=True)
bk3['length']=bk3['itemsets'].apply(lambda x:len(x))
bk10=bk3[bk3['length']==1].copy()
bk10['Item#1']=pd.DataFrame(bk10['itemsets'].tolist(),index=bk10.index)
bk11=bk10[['Item#1','support']].copy()
bk11=bk11.reset_index(drop=True).rename(columns={'support':'Item#1support'})
bk11.head(5)

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,Item#1,Item#1support
0,1010,0.009817
1,1017,0.006559
2,1050,0.006311
3,1051,0.005115
4,11460,0.007507


In [ ]:
bk3=bk3[bk3['length']==2]
bk3['Item#1'],bk3['Item#2']=zip(*bk3.itemsets)
bk3.head(5)

,support,itemsets,length,Item#1,Item#2
175,0.000082,"(1010, 1017)",2,1010,1017
176,0.000041,"(1010, 1051)",2,1010,1051
177,0.000206,"(1150, 1010)",2,1150,1010
178,0.000041,"(1151, 1010)",2,1151,1010
179,0.000165,"(11560, 1010)",2,11560,1010


In [ ]:
bk4=bk3.copy()
bk4['Item#2'],bk4['Item#1']=zip(*bk4.itemsets)

bk5=pd.concat([bk3,bk4])
bk5['Occurrence']=bk5['support']*len(bk2.index)
bk5=bk5.reset_index(drop=True)
bk12=bk5.merge(bk11,how='left',on='Item#1')
bk12.head(5)

,support,itemsets,length,Item#1,Item#2,Occurrence,Item#1support
0,0.000082,"(1010, 1017)",2,1010,1017,2.0,0.009817
1,0.000041,"(1010, 1051)",2,1010,1051,1.0,0.009817
2,0.000206,"(1150, 1010)",2,1150,1010,5.0,0.013777
3,0.000041,"(1151, 1010)",2,1151,1010,1.0,0.010436
4,0.000165,"(11560, 1010)",2,11560,1010,4.0,0.009859


In [ ]:
bk12['%Item#1&Item#2/Item#1']=(bk12['support']/bk12['Item#1support'])
bk13=bk12[['Item#1','Item#2','%Item#1&Item#2/Item#1','Occurrence']].copy()
bk13=bk13.sort_values(by=['Item#1','%Item#1&Item#2/Item#1'],ascending=[True,False])
bk13['%Item#1&Item#2/Item#1']=bk13['%Item#1&Item#2/Item#1'].astype(float).map(lambda n: '{:.4%}'.format(n))
bk13=bk13.groupby('Item#1').head(10).reset_index(drop=True)
bk13['Occurrence'] = bk13['Occurrence'].astype(int)
bk13.head(20)

,Item#1,Item#2,%Item#1&Item#2/Item#1,Occurrence
0,1010,2010,23.5294%,56
1,1010,7770,18.9076%,45
2,1010,2900,10.9244%,26
3,1010,8030,10.5042%,25
4,1010,1350,9.2437%,22
5,1010,16002,8.4034%,20
6,1010,6001,8.4034%,20
7,1010,8010,7.5630%,18
8,1010,17760,5.8824%,14
9,1010,7610,5.8824%,14


In [ ]:
import pandas as pd
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Convert the Pandas DataFrame to a Spark DataFrame
spark_df = spark.createDataFrame(bk13)

# Register the DataFrame as a temporary table
spark_df.createOrReplaceTempView("temp_table")

# Define the database and view names
database_name = "dtc"
view_name = "MBA"

# Drop the old view if it exists
spark.sql(f"DROP TABLE IF EXISTS {database_name}.{view_name}")

# Create a persistent table from the temporary table
spark.sql(f"CREATE TABLE {database_name}.{view_name} USING DELTA AS SELECT * FROM temp_table")

# Save the DataFrame as a CSV file
result_df_filename=f"MBA_output_{current_date}.csv"
result_tb_path = f"/mnt/output tables for ml projects/MBA/{result_df_filename}"
spark_df.write.csv(result_tb_path, header=True, mode="overwrite")

# Remove the temporary table
spark.catalog.dropTempView("temp_table")

# Get a list of files in the output folder
path2 = "dbfs:/mnt/output tables for ml projects/MBA/"
file_list2 = dbutils.fs.ls(path2)

# Sort the files by modification time (most recent first)
sorted_files2 = sorted(file_list2, key=lambda x: x.modificationTime, reverse=True)

# Keep only the latest five files
latest_files2 = sorted_files2[:5]

# Delete files that are not in the latest five
for file in file_list2:
    if file not in latest_files2:
        dbutils.fs.rm(file.path)

/databricks/spark/python/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/databricks/spark/python/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/databricks/spark/python/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/databricks/spark/python/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/databricks/spark/python/pyspark/sql/pandas/utils.py:64: DeprecationWarning: distutils V

In [ ]:
%sql

--- Test View Table
select *
from dtc.item_pairing
limit 10

Item#1,Item#2,%Item#1&Item#2/Item#1,Occurrence
1010,2010,23.5294%,56
1010,7770,18.9076%,45
1010,2900,10.9244%,26
1010,8030,10.5042%,25
1010,1350,9.2437%,22
1010,16002,8.4034%,20
1010,6001,8.4034%,20
1010,8010,7.5630%,18
1010,17760,5.8824%,14
1010,7610,5.8824%,14
